# Architecture 2: RAG Standard (Retrieval-Augmented Generation)

Cette architecture implémente un système RAG qui sépare la capacité de raisonnement (LLM) de la base de connaissances factuelles (ChromaDB).

## Caractéristiques principales:
- **Base vectorielle**: ChromaDB
- **Embedding**: paraphrase-multilingual-mpnet-base-v2 (768 dimensions)
- **Chunking**: 512 tokens max, 50 tokens overlap
- **LLM**: Llama 3.2 3B (peut être utilisé sans fine-tuning ou avec)
- **Sources**: FAQ, procédures, documentation opérateurs, conversations historiques

<a href="https://colab.research.google.com/github/AmedBah/memoire/blob/main/notebooks/architecture_2/02_architecture_2_rag_standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 Configuration pour Google Colab

**Note**: Cette section est spécifique à Google Colab. Si vous exécutez ce notebook localement, vous pouvez ignorer ces cellules.

In [ ]:
# Vérifier le type de runtime
import os
import sys

# Détecter si on est sur Colab
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("✓ Exécution sur Google Colab")
    
    # Vérifier le type de GPU
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU détecté: {gpu_name}")
        print(f"✓ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Recommandations selon le GPU
        if 'T4' in gpu_name:
            print("\n⚠️  GPU T4 détecté (16 GB): Convient pour ce notebook mais les temps d'entraînement seront plus longs")
        elif 'V100' in gpu_name or 'A100' in gpu_name:
            print(f"\n✓ {gpu_name}: Parfait pour ce notebook!")
    else:
        print("\n❌ ATTENTION: Aucun GPU détecté!")
        print("   Pour activer le GPU: Runtime > Change runtime type > GPU")
        print("   Pour Colab Pro: Choisir 'High-RAM' ou 'Premium GPU'")
else:
    print("✓ Exécution locale")
    import torch
    if torch.cuda.is_available():
        print(f"✓ GPU disponible: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  Aucun GPU détecté - l'entraînement sera très lent")

In [ ]:
# Monter Google Drive (uniquement sur Colab)
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Définir le chemin vers les données
    # OPTION 1: Données dans Google Drive
    # DATA_DIR = '/content/drive/MyDrive/memoire/data'
    
    # OPTION 2: Cloner le repo et utiliser les données locales
    print("\n📥 Clonage du repository...")
    !git clone https://github.com/AmedBah/memoire.git /content/memoire
    DATA_DIR = '/content/memoire/data'
    
    print(f"\n✓ Répertoire de données: {DATA_DIR}")
    
    # Vérifier que les données sont présentes
    if os.path.exists(DATA_DIR):
        print("✓ Données trouvées!")
        !ls -lh {DATA_DIR}
    else:
        print(f"❌ ERREUR: Répertoire {DATA_DIR} non trouvé!")
        print("   Veuillez soit:")
        print("   1. Copier le dossier 'data' dans votre Google Drive")
        print("   2. Ou le repository sera cloné automatiquement")
else:
    # Exécution locale
    DATA_DIR = '../../data'
    print(f"✓ Répertoire de données local: {DATA_DIR}")

In [ ]:
# Fonction helper pour obtenir les chemins de données
def get_data_path(relative_path):
    """Obtenir le chemin absolu d'un fichier de données"""
    return os.path.join(DATA_DIR, relative_path)

# Exemples de chemins
print("Chemins de données configurés:")
print(f"  Conversations: {get_data_path('conversations/conversation_1000_finetune.jsonl')}")
print(f"  FAQs: {get_data_path('faqs/faq_easytransfert.json')}")
print(f"  Opérateurs: {get_data_path('operators/operators_info.json')}")
print(f"  Procédures: {get_data_path('procedures/procedures_resolution.json')}")
print(f"  Expressions: {get_data_path('expressions/expressions_ivoiriennes.json')}")
print(f"  Documents: {get_data_path('documents/doc.txt.txt')}")

## 1. Installation et Configuration

In [ ]:
# Installation des dépendances
!pip install -q chromadb sentence-transformers langchain langchain-community transformers torch

import os
import json
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
from typing import List, Dict, Tuple
import time

print(f"GPU disponible: {torch.cuda.is_available()}")

## 2. Configuration ChromaDB et Modèle d'Embedding

In [ ]:
# Initialiser ChromaDB
chroma_client = chromadb.PersistentClient(path="./chromadb_easytransfert")

# Charger le modèle d'embedding multilingue
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

print(f"Dimension des embeddings: {embedding_model.get_sentence_embedding_dimension()}")
print(f"Performance estimée: ~2000 phrases/seconde sur CPU")

# Créer ou récupérer la collection
collection = chroma_client.get_or_create_collection(
    name="easytransfert_knowledge_base",
    metadata={"description": "Base de connaissances EasyTransfert pour RAG"}
)

print(f"Collection créée/récupérée: {collection.name}")
print(f"Nombre de documents existants: {collection.count()}")

## 3. Préparation des Données et Chunking

Stratégie de chunking adaptative:
- Taille maximale: 512 tokens (~400 mots français)
- Chevauchement: 50 tokens (10%)
- Séparateurs hiérarchiques: double saut de ligne, point, virgule
- Métadonnées enrichies: catégorie, opérateur, mots-clés, date

In [ ]:
# Configuration du text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", ", ", " ", ""]
)

def create_chunks_with_metadata(documents: List[Dict]) -> Tuple[List[str], List[Dict], List[str]]:
    """
    Crée des chunks avec métadonnées enrichies
    
    Args:
        documents: Liste de documents avec contenu et métadonnées
    
    Returns:
        Tuple de (chunks, metadatas, ids)
    """
    all_chunks = []
    all_metadatas = []
    all_ids = []
    
    for doc_idx, doc in enumerate(documents):
        # Découper le texte
        chunks = text_splitter.split_text(doc["content"])
        
        for chunk_idx, chunk in enumerate(chunks):
            all_chunks.append(chunk)
            
            # Créer les métadonnées
            metadata = {
                "category": doc.get("category", "unknown"),
                "operator": doc.get("operator", "Tous"),
                "source": doc.get("source", "internal"),
                "chunk_index": chunk_idx,
                "total_chunks": len(chunks),
            }
            
            # Ajouter les mots-clés si disponibles
            if "keywords" in doc:
                metadata["keywords"] = ",".join(doc["keywords"])
            
            all_metadatas.append(metadata)
            all_ids.append(f"doc_{doc_idx}_chunk_{chunk_idx}")
    
    return all_chunks, all_metadatas, all_ids

print("Fonction de chunking définie.")

## 4. Création de la Base de Connaissances

Sources de données:
1. FAQ EasyTransfert
2. Procédures opérationnelles
3. Documentation des opérateurs
4. Conversations historiques réussies

In [ ]:
# Exemples de documents de connaissances
knowledge_base_documents = [
    # FAQ
    {
        "content": """EasyTransfert permet de transférer de l'argent entre différents opérateurs mobiles en Côte d'Ivoire. 
        Les opérateurs supportés sont: MTN Mobile Money, Orange Money, Moov Money, Wave et Trésor Money. 
        Pour effectuer un transfert, ouvrez l'application, sélectionnez l'opérateur d'envoi et de réception, 
        entrez le numéro du destinataire et le montant, puis confirmez la transaction.""",
        "category": "faq",
        "operator": "Tous",
        "source": "FAQ officielle",
        "keywords": ["transfert", "opérateurs", "fonctionnement"]
    },
    {
        "content": """Les frais de transaction EasyTransfert varient selon le montant et les opérateurs concernés.
        En général, les frais sont de 1% à 2% du montant transféré, avec un minimum de 25 FCFA et un maximum de 500 FCFA.
        Les transferts entre certains opérateurs peuvent bénéficier de tarifs promotionnels.""",
        "category": "faq",
        "operator": "Tous",
        "source": "FAQ officielle",
        "keywords": ["frais", "tarifs", "coût"]
    },
    
    # Procédures
    {
        "content": """Procédure de résolution pour transaction non aboutie:
        1. Demander à l'utilisateur l'identifiant EasyTransfert (commence par EFB.)
        2. Demander l'identifiant de l'opérateur d'envoi (format varie selon opérateur)
        3. Vérifier le statut de la transaction dans le système
        4. Si la transaction est bloquée, initier un remboursement
        5. Si le problème persiste, escalader vers le service technique
        Contact service client: 2522018730 (WhatsApp 24h/24)""",
        "category": "procedure",
        "operator": "Tous",
        "source": "Manuel de procédures",
        "keywords": ["transaction échouée", "remboursement", "résolution"]
    },
    
    # Documentation opérateurs
    {
        "content": """Format des identifiants de transaction par opérateur:
        - EasyTransfert: EFB.XXXXXXXXX (commence toujours par EFB.)
        - MTN Mobile Money: Série de chiffres uniquement, généralement 10 chiffres
        - Orange Money (envoi): MP suivi de 10 chiffres (ex: MP1234567890)
        - Moov Money (envoi): MRCH ou CF suivi de caractères alphanumériques
        - Wave: Format variable, souvent commence par T suivi de chiffres
        - Trésor Money: Format variable selon la transaction""",
        "category": "operator_info",
        "operator": "Tous",
        "source": "Documentation technique",
        "keywords": ["identifiants", "formats", "référence transaction"]
    },
    {
        "content": """Limites de transaction par opérateur:
        - MTN: Minimum 100 FCFA, Maximum 2 000 000 FCFA par transaction
        - Orange: Minimum 100 FCFA, Maximum 1 500 000 FCFA par transaction
        - Moov: Minimum 100 FCFA, Maximum 1 000 000 FCFA par transaction
        - Wave: Minimum 100 FCFA, Maximum 5 000 000 FCFA par transaction
        - Trésor Money: Minimum 100 FCFA, Maximum 1 000 000 FCFA par transaction""",
        "category": "operator_info",
        "operator": "Tous",
        "source": "Documentation technique",
        "keywords": ["limites", "montants", "maximum", "minimum"]
    },
    
    # Problèmes courants
    {
        "content": """Mot de passe oublié - Procédure de réinitialisation:
        1. Ouvrir l'application EasyTransfert
        2. Cliquer sur "Mot de passe oublié" sur l'écran de connexion
        3. Entrer votre numéro de téléphone enregistré
        4. Vous recevrez un code de vérification par SMS
        5. Entrer le code reçu
        6. Créer un nouveau mot de passe (minimum 8 caractères)
        Si vous ne recevez pas le SMS, contactez le 2522018730""",
        "category": "procedure",
        "operator": "Tous",
        "source": "Guide utilisateur",
        "keywords": ["mot de passe", "réinitialisation", "connexion"]
    },
]

print(f"Nombre de documents à indexer: {len(knowledge_base_documents)}")

# Créer les chunks
chunks, metadatas, ids = create_chunks_with_metadata(knowledge_base_documents)

print(f"Nombre de chunks créés: {len(chunks)}")
print(f"\nExemple de chunk:")
print(f"Contenu: {chunks[0][:200]}...")
print(f"Métadonnées: {metadatas[0]}")

## 5. Vectorisation et Indexation dans ChromaDB

In [ ]:
# Générer les embeddings
print("Génération des embeddings...")
start_time = time.time()

embeddings = embedding_model.encode(chunks, show_progress_bar=True, convert_to_numpy=True)

embedding_time = time.time() - start_time
print(f"Temps de vectorisation: {embedding_time:.2f}s")
print(f"Performance: {len(chunks) / embedding_time:.1f} chunks/seconde")

# Ajouter à la collection ChromaDB
if collection.count() == 0:  # Éviter les doublons
    collection.add(
        embeddings=embeddings.tolist(),
        documents=chunks,
        metadatas=metadatas,
        ids=ids
    )
    print(f"\n{len(chunks)} chunks indexés dans ChromaDB")
else:
    print(f"\nCollection existe déjà avec {collection.count()} documents")

print(f"Base de connaissances prête!")

## 6. Fonction de Récupération (Retrieval)

Phase de récupération (~150-200ms):
- Vectorisation de la question
- Recherche de similarité cosinus
- Sélection des top-k chunks (défaut: 3)
- Filtrage par score (> 0.5)

In [ ]:
def retrieve_context(query: str, top_k: int = 3, min_score: float = 0.5) -> List[Dict]:
    """
    Récupère les chunks les plus pertinents pour une requête
    
    Args:
        query: Question de l'utilisateur
        top_k: Nombre de chunks à retourner
        min_score: Score minimum de pertinence (similarité cosinus)
    
    Returns:
        Liste de dictionnaires contenant chunks, scores et métadonnées
    """
    # Vectoriser la requête
    query_embedding = embedding_model.encode([query])[0]
    
    # Recherche dans ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k,
    )
    
    # Formater les résultats
    retrieved_docs = []
    
    for i in range(len(results['documents'][0])):
        # Calculer le score de similarité (distance cosinus -> similarité)
        distance = results['distances'][0][i]
        similarity = 1 - distance  # ChromaDB retourne la distance, on veut la similarité
        
        if similarity >= min_score:
            retrieved_docs.append({
                "content": results['documents'][0][i],
                "metadata": results['metadatas'][0][i],
                "score": similarity,
                "id": results['ids'][0][i]
            })
    
    return retrieved_docs

# Test de récupération
test_query = "Comment faire un transfert de MTN vers Orange ?"
print(f"Requête de test: {test_query}\n")

start_time = time.time()
retrieved = retrieve_context(test_query, top_k=3)
retrieval_time = time.time() - start_time

print(f"Temps de récupération: {retrieval_time * 1000:.0f}ms\n")
print(f"Nombre de documents récupérés: {len(retrieved)}\n")

for i, doc in enumerate(retrieved, 1):
    print(f"Document {i} (score: {doc['score']:.3f})")
    print(f"Catégorie: {doc['metadata']['category']}")
    print(f"Contenu: {doc['content'][:200]}...\n")

## 7. Chargement du LLM pour la Génération

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuration pour quantification 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Charger le modèle Llama 3.2 3B
model_name = "meta-llama/Llama-3.2-3B-Instruct"
# Ou utiliser le modèle fine-tuné de l'Architecture 1 si disponible
# model_name = "./models/architecture1_lora"

print(f"Chargement du modèle {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print("Modèle chargé avec succès!")

## 8. Prompt RAG Augmenté

Format structuré avec:
- Section contexte récupéré (documents + métadonnées)
- Instructions strictes contre hallucinations
- Comportements attendus (empathie, citations)

In [ ]:
def build_rag_prompt(query: str, retrieved_docs: List[Dict]) -> str:
    """
    Construit un prompt RAG structuré avec contexte et instructions
    """
    
    system_prompt = """Tu es un assistant intelligent du service client EasyTransfert.

RÈGLES IMPORTANTES:
- Réponds UNIQUEMENT en te basant sur le contexte fourni ci-dessous
- Ne JAMAIS inventer d'informations
- Si l'information n'est pas dans le contexte, dis "Je n'ai pas cette information, contactez le 2522018730"
- Cite les sources quand c'est pertinent
- Utilise un ton chaleureux avec des émojis appropriés 🤗😊
- Mentionne régulièrement EasyTransfert
"""
    
    # Construire le contexte récupéré
    context_section = "CONTEXTE RÉCUPÉRÉ:\n\n"
    
    for i, doc in enumerate(retrieved_docs, 1):
        context_section += f"Document {i} (Pertinence: {doc['score']:.2f})\n"
        context_section += f"Source: {doc['metadata'].get('source', 'interne')}\n"
        context_section += f"Catégorie: {doc['metadata']['category']}\n"
        context_section += f"Contenu: {doc['content']}\n\n"
    
    # Construire le prompt complet au format Llama 3
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{context_section}
QUESTION: {query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    return prompt

# Test
test_prompt = build_rag_prompt(test_query, retrieved)
print("Exemple de prompt RAG:")
print("=" * 80)
print(test_prompt[:1000] + "...")
print("=" * 80)

## 9. Pipeline RAG Complet

Intégration Récupération + Génération

In [ ]:
def rag_chat(query: str, top_k: int = 3, min_score: float = 0.5) -> Dict:
    """
    Pipeline RAG complet: Récupération + Génération
    
    Returns:
        Dict avec response, sources, et métriques de performance
    """
    # Phase 1: Récupération
    retrieval_start = time.time()
    retrieved_docs = retrieve_context(query, top_k, min_score)
    retrieval_time = time.time() - retrieval_start
    
    if not retrieved_docs:
        return {
            "response": "Je n'ai pas trouvé d'information pertinente dans ma base de connaissances. Contactez le service client EasyTransfert au 2522018730 pour une assistance personnalisée 🤗",
            "sources": [],
            "retrieval_time_ms": retrieval_time * 1000,
            "generation_time_ms": 0,
            "total_time_ms": retrieval_time * 1000
        }
    
    # Construire le prompt
    prompt = build_rag_prompt(query, retrieved_docs)
    
    # Phase 2: Génération
    generation_start = time.time()
    
    inputs = tokenizer([prompt], return_tensors="pt").to(llm_model.device)
    
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    
    generation_time = time.time() - generation_start
    
    # Extraire les sources
    sources = [
        {
            "category": doc["metadata"]["category"],
            "source": doc["metadata"].get("source", "interne"),
            "score": doc["score"]
        }
        for doc in retrieved_docs
    ]
    
    return {
        "response": response.strip(),
        "sources": sources,
        "retrieval_time_ms": retrieval_time * 1000,
        "generation_time_ms": generation_time * 1000,
        "total_time_ms": (retrieval_time + generation_time) * 1000,
        "num_retrieved_docs": len(retrieved_docs)
    }

print("Pipeline RAG défini.")

## 10. Tests et Évaluation

In [ ]:
# Tests
test_queries = [
    "Comment faire un transfert entre opérateurs ?",
    "Quels sont les frais de transaction ?",
    "Mon transfert n'est pas arrivé, que faire ?",
    "J'ai oublié mon mot de passe",
    "Quel est le format de l'identifiant Orange ?",
    "Quelle est la limite de transfert pour Wave ?"
]

print("=" * 80)
print("TESTS DU SYSTÈME RAG")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\nTest {i}: {query}")
    print("-" * 80)
    
    result = rag_chat(query)
    
    print(f"Réponse: {result['response']}")
    print(f"\nPerformance:")
    print(f"  - Récupération: {result['retrieval_time_ms']:.0f}ms")
    print(f"  - Génération: {result['generation_time_ms']:.0f}ms")
    print(f"  - Total: {result['total_time_ms']:.0f}ms")
    print(f"  - Documents récupérés: {result['num_retrieved_docs']}")
    print(f"\nSources utilisées:")
    for src in result['sources']:
        print(f"  - {src['category']} (score: {src['score']:.2f})")
    print("=" * 80)

## 11. Métriques de Performance

In [ ]:
import numpy as np

# Collecter les métriques
metrics = {
    "retrieval_times": [],
    "generation_times": [],
    "total_times": [],
    "num_docs_retrieved": []
}

for query in test_queries:
    result = rag_chat(query)
    metrics["retrieval_times"].append(result["retrieval_time_ms"])
    metrics["generation_times"].append(result["generation_time_ms"])
    metrics["total_times"].append(result["total_time_ms"])
    metrics["num_docs_retrieved"].append(result["num_retrieved_docs"])

print("\nMÉTRIQUES DE PERFORMANCE - ARCHITECTURE 2 (RAG)")
print("=" * 80)
print(f"Récupération:")
print(f"  - Moyenne: {np.mean(metrics['retrieval_times']):.0f}ms")
print(f"  - Min/Max: {np.min(metrics['retrieval_times']):.0f}ms / {np.max(metrics['retrieval_times']):.0f}ms")
print(f"\nGénération:")
print(f"  - Moyenne: {np.mean(metrics['generation_times']):.0f}ms")
print(f"  - Min/Max: {np.min(metrics['generation_times']):.0f}ms / {np.max(metrics['generation_times']):.0f}ms")
print(f"\nTemps total:")
print(f"  - Moyenne: {np.mean(metrics['total_times']):.0f}ms")
print(f"  - Écart-type: {np.std(metrics['total_times']):.0f}ms")
print(f"\nDocuments récupérés:")
print(f"  - Moyenne: {np.mean(metrics['num_docs_retrieved']):.1f}")
print("=" * 80)

## 12. Avantages et Limites de l'Architecture 2

### Avantages:
- ✅ **Véracité**: Réponses ancrées dans sources vérifiables
- ✅ **Traçabilité**: Citations des sources utilisées
- ✅ **Actualisation**: Ajout de documents sans réentraînement
- ✅ **Réduction hallucinations**: Contexte factuel fourni au LLM
- ✅ **Transparence**: Scores de pertinence et métadonnées

### Limites:
- ❌ **Latence**: ~2-3s (récupération + génération)
- ❌ **Complexité**: Pipeline multi-composants
- ❌ **Dépendances**: ChromaDB + LLM + embedding model
- ❌ **Requêtes complexes**: Pas de raisonnement multi-étapes
- ❌ **Données temps réel**: Pas d'accès aux bases transactionnelles